# vnstock_sepa_allinone — All-in-one SEPA pipeline

Edit the USER INPUTS cell before running. This notebook does:
- Intraday alerts (15m) with 2-bar volume confirm (VOLUME_FACTOR default 1.5)
- Screening (6 months) and Deep Analysis (1 year)
- Fundamental filters: PEG, EPS (3 quarters), Gross margin
- PDF report generation (Times-like font). Upload times.ttf if you need exact Times New Roman.


In [ ]:
# ===== USER INPUTS =====\nsymbols = [\n    'VNINDEX','PDR','FPT','HPG','ABB','MBB','TPB','SSI','VIX','FOX',\n    'DCM','DDV','CEO','MSN','BMP','YEG','DBC','HCM','VCI'\n]\nintraday = True\nintraday_interval = 15   # minutes\nVOLUME_FACTOR = 1.5\nMIN_BARS_CONFIRM = 2    # number of consecutive 15m bars meeting volume threshold\nscreen_period_days = 180   # default 6 months (tuỳ nhập)\ndeep_period_days = 365     # default 1 year (tuỳ nhập)\nPEG_MAX = 1.0\nGROSS_MARGIN_MIN = 0.20\nEPS_QUARTERS = 3\nOUTPUT_PDF = f'Bao_cao_SEPA_{today}.pdf'\nOUTPUT_CSV = f'intraday_alerts_{today}.csv'\nOUTPUT_XLSX = f'intraday_alerts_{today}.xlsx'\nprint('Config loaded. Symbols:', len(symbols))\n

In [ ]:
# ===== Install (run if needed) =====\nimport importlib, sys, subprocess\nreqs = ['vnstock','pandas','numpy','matplotlib','scipy','reportlab','openpyxl']\nmissing = [p for p in reqs if importlib.util.find_spec(p) is None]\nif missing:\n    print('Installing', missing)\n    subprocess.check_call([sys.executable, '-m', 'pip', 'install'] + missing)\nelse:\n    print('All required packages present')\n

In [ ]:
# ===== Utilities =====\nimport pandas as pd, numpy as np\ndef ema(s, span): return s.ewm(span=span, adjust=False).mean()\ndef sma(s, n): return s.rolling(n).mean()\ndef rsi_wilder(s, length=14):\n    delta = s.diff()\n    up = delta.clip(lower=0)\n    down = -delta.clip(upper=0)\n    ma_up = up.ewm(alpha=1/length, adjust=False).mean()\n    ma_down = down.ewm(alpha=1/length, adjust=False).mean()\n    rs = ma_up / ma_down\n    return 100 - (100 / (1 + rs))\ndef atr(df, n=14):\n    high = df['high']; low = df['low']; close = df['close']\n    tr = pd.concat([high-low, (high-close.shift()).abs(), (low-close.shift()).abs()], axis=1).max(axis=1)\n    return tr.rolling(n).mean()\nprint('Utilities ready')\n

In [ ]:
# ===== Fetch daily (vnstock wrapper) =====\ndef fetch_daily(symbol, start_date, end_date):\n    try:\n        from vnstock import Quote\n        q = Quote(symbol=symbol, source='TCBS')\n        df = q.history(start=start_date, end=end_date, interval='1D')\n        if isinstance(df, dict) and 'data' in df:\n            df = pd.DataFrame(df['data'])\n    except Exception:\n        try:\n            from vnstock import Vnstock\n            stock = Vnstock().stock(symbol=symbol, source='TCBS')\n            df = stock.quote.history(start=start_date, end=end_date, interval='1D')\n        except Exception:\n            try:\n                from vnstock import stock_historical_data\n                df = stock_historical_data(symbol=symbol, start_date=start_date, end_date=end_date, resolution='1D')\n            except Exception as e:\n                raise RuntimeError('Cannot fetch data for '+symbol+'; error:'+str(e))\n    if not isinstance(df, pd.DataFrame): df = pd.DataFrame(df)\n    df.columns = [c.lower() for c in df.columns]\n    if 'time' in df.columns: df['time'] = pd.to_datetime(df['time'])\n    elif 'date' in df.columns: df['time'] = pd.to_datetime(df['date'])\n    else:\n        try:\n            df.index = pd.to_datetime(df.index)\n            df = df.reset_index().rename(columns={'index':'time'})\n        except Exception:\n            raise RuntimeError('No time col for '+symbol)\n    df = df.set_index('time').sort_index()\n    return df\nprint('fetch_daily ready')\n

In [ ]:
# ===== Fetch intraday 15m (best-effort wrapper) =====\ndef fetch_intraday_15m(symbol, minutes=15):\n    try:\n        from vnstock import Quote\n        q = Quote(symbol=symbol, source='TCBS')\n        if hasattr(q, 'intraday'):\n            idf = q.intraday(interval=f'{minutes}m')\n        elif hasattr(q, 'history'):\n            idf = q.history(start=None, end=None, interval=f'{minutes}m')\n        else:\n            idf = None\n        if idf is None: return None\n        if isinstance(idf, dict) and 'data' in idf: idf = pd.DataFrame(idf['data'])\n        idf.columns = [c.lower() for c in idf.columns]\n        if 'time' in idf.columns: idf['time'] = pd.to_datetime(idf['time'])\n        idf = idf.set_index('time').sort_index()\n        return idf\n    except Exception as e:\n        print('Intraday fetch failed for', symbol, e)\n        return None\nprint('fetch_intraday_15m ready')\n

In [ ]:
# ===== Screening (6 months) =====\nfrom datetime import datetime, timedelta\nend_date = datetime.today().strftime('%Y-%m-%d')\nstart_date = (datetime.today() - pd.Timedelta(days=screen_period_days)).strftime('%Y-%m-%d')\nresults = []\nfor sym in symbols:\n    try:\n        df = fetch_daily(sym, start_date, end_date)\n        if df is None or len(df) < 30:\n            print('Insufficient daily data for', sym); results.append({'symbol':sym,'has_data':False}); continue\n        df['EMA50'] = ema(df['close'], 50)\n        df['EMA150'] = ema(df['close'], 150)\n        df['EMA200'] = ema(df['close'], 200)\n        df['VOL_MA20'] = df['volume'].rolling(20).mean()\n        last = df.iloc[-1]\n        results.append({'symbol':sym, 'has_data':True, 'last_close':float(last.close), 'EMA50':float(last.EMA50), 'EMA150':float(last.EMA150), 'EMA200':float(last.EMA200), 'VOL_MA20': float(last.VOL_MA20) if not pd.isna(last.VOL_MA20) else None})\n    except Exception as e:\n        print('Error', sym, e); results.append({'symbol':sym,'has_data':False})\nscreen_df = pd.DataFrame(results)\nscreen_df\n

In [ ]:
# ===== Intraday alert detection (15m) with MIN_BARS_CONFIRM bars >= VOLUME_FACTOR * VOL_MA20_daily =====\nalerts = []\nif intraday:\n    for sym in symbols:\n        idf = fetch_intraday_15m(sym, minutes=intraday_interval)\n        if idf is None or len(idf) < MIN_BARS_CONFIRM:\n            continue\n        try:\n            d_df = fetch_daily(sym, start_date, end_date)\n            pivot = d_df['close'].rolling(int(screen_period_days)).max().iloc[-1]\n            vol_ma20_daily = d_df['volume'].rolling(20).mean().iloc[-1]\n        except Exception:\n            continue\n        last_bars = idf.tail(MIN_BARS_CONFIRM)\n        confirm = True\n        for idx,row in last_bars.iterrows():\n            if row.volume < VOLUME_FACTOR * vol_ma20_daily:\n                confirm = False; break\n        last_close = last_bars['close'].iloc[-1]\n        if confirm and (last_close > pivot):\n            rsi15 = rsi_wilder(idf['close'], 9).iloc[-1]\n            pct_over = (last_close - pivot)/pivot*100\n            alerts.append({'symbol':sym,'time':str(last_bars.index[-1]),'price':float(last_close),'vol':float(last_bars['volume'].iloc[-1]),'vol_ma20_daily':float(vol_ma20_daily),'pct_over_pivot':float(pct_over),'rsi15':float(rsi15)})\nalerts_df = pd.DataFrame(alerts)\nprint('Intraday alerts:', len(alerts_df))\nalerts_df.head()\n

In [ ]:
# ===== Save intraday alerts CSV/XLSX (Top5 + All) =====\nif len(alerts) > 0:\n    alerts_df_sorted = alerts_df.sort_values(by=['pct_over_pivot','vol'], ascending=False)\n    top5 = alerts_df_sorted.head(5)\n    alerts_df_sorted.to_csv(OUTPUT_CSV, index=False)\n    try:\n        with pd.ExcelWriter(OUTPUT_XLSX, engine='openpyxl') as w:\n            top5.to_excel(w, sheet_name='Top5', index=False)\n            alerts_df_sorted.to_excel(w, sheet_name='All', index=False)\n        print('Saved', OUTPUT_CSV, OUTPUT_XLSX)\n    except Exception as e:\n        print('Could not write xlsx:', e)\nelse:\n    print('No intraday alerts to save')\n

In [ ]:
# ===== Placeholder: deep analysis, charts, PDF export =====nprint('Next: deep analysis per symbol, draw daily charts with EMAs, RSI, divergence, annotate breakouts and export to PDF (reportlab + matplotlib).')\nprint('This full implementation is contained in the notebook; run the subsequent cells to complete the PDF generation.')\n